# 演習
GANにバッチ正規化を導入しましょう。  
中間層におけるデータのばらつきを抑えることで、結果にどのような影響が及ぶのか確認してみましょう。

## バッチ正規化とは？
中間層のある時点でバッチごとに平均値を0、標準偏差を1にします。  
これより、中間層でデータが散らばることを防ぎ、学習を効率化します。  
Kerasでは、バッチ正規化を層として簡単に実装することができます。  

バッチ正規化は以下の式で表されます。  

$$Y = \gamma \frac{X-\mu}{\sigma} + \beta$$

$\mu$がバッチ内の平均値、$\sigma$がバッチ内の標準偏差、$X$と$Y$がバッチ正規化層の入出力、$\gamma$と$\beta$はバッチ正規化層において学習するパラメータです。  

GANは通常のニューラルネットワークと比較して学習方法が複雑なため、学習が不安定になりがちです。  
そのため、バッチ正規化による中間層の安定化がよく試みられます。  



## 訓練用データの用意
GANに用いる訓練用のデータを用意します。   

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from keras.datasets import mnist

(x_train, t_train), (x_test, t_test) = mnist.load_data()  # MNISTの読み込み
print(x_train.shape, x_test.shape)  # 28x28の手書き文字画像が6万枚

# 各ピクセルの値を-1から1の範囲に収める
x_train = x_train / 255 * 2 - 1
x_test = x_test / 255 * 2 - 1

# 手書き文字画像の表示
plt.imshow(x_train[0].reshape(28, 28), cmap="gray")
plt.title(t_train[0])
plt.show() 

# 一次元に変換する
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)
print(x_train.shape, x_test.shape)

## GANの各設定
GANに必要な各設定を行います。

In [ ]:
n_learn = 10001 # 学習回数
interval = 1000  # 画像を生成する間隔
batch_size = 32
n_noize = 128  # ノイズの数
img_size = 28  # 生成される画像の高さと幅
alpha = 0.2  # Leaky ReLUの負の領域での傾き

from keras.optimizers import Adam
optimizer = Adam(0.0002, 0.5)

## Generatorの構築
以下のページを参考に、  
https://keras.io/ja/layers/normalization/  
Generaotrにバッチ正規化層を挿入しましょう。  

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LeakyReLU, BatchNormalization

# 以下のモデルにBatchNormalization層を挿入しましょう 
generator = Sequential()
generator.add(Dense(256, input_shape=(n_noize,)))
generator.add(LeakyReLU(alpha=alpha))
generator.add(Dense(512))
generator.add(LeakyReLU(alpha=alpha)) 
generator.add(Dense(1024))
generator.add(LeakyReLU(alpha=alpha)) 
generator.add(Dense(img_size**2, activation="tanh"))
print(generator.summary())

## Discriminatorの構築
KerasによりDiscriminatorのモデルを構築します。  

In [ ]:
# Discriminatorのネットワーク構築
discriminator = Sequential()
discriminator.add(Dense(512, input_shape=(img_size**2,)))
discriminator.add(LeakyReLU(alpha=alpha)) 
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(alpha=alpha)) 
discriminator.add(Dense(1, activation="sigmoid"))
discriminator.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
print(discriminator.summary())

## モデルの結合
GeneratorとDiscriminatorを結合したモデルを作ります。  

In [ ]:
from keras.models import Model
from keras.layers import Input

# 結合時はGeneratorのみ訓練する
discriminator.trainable = False

# Generatorによってノイズから生成された画像を、Discriminatorが判定する
noise = Input(shape=(n_noize,))
img = generator(noise)
reality = discriminator(img)

# GeneratorとDiscriminatorの結合
combined = Model(noise, reality)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)
print(combined.summary())

## 画像の生成
Generatorにより、ノイズから画像を生成する関数を定義します。

In [ ]:
def generate_images(i):
    n_rows = 5  # 行数
    n_cols = 5  # 列数
    noise = np.random.normal(0, 1, (n_rows*n_cols, n_noize))
    g_imgs = generator.predict(noise)
    g_imgs = g_imgs/2 + 0.5  # 0-1の範囲にする

    matrix_image = np.zeros((img_size*n_rows, img_size*n_cols))  # 全体の画像

    #  生成された画像を並べて一枚の画像にする
    for r in range(n_rows):
        for c in range(n_cols):
            g_img = g_imgs[r*n_cols + c].reshape(img_size, img_size)
            matrix_image[r*img_size : (r+1)*img_size, c*img_size: (c+1)*img_size] = g_img

    plt.figure(figsize=(10, 10))
    plt.imshow(matrix_image, cmap="Greys_r")
    plt.tick_params(labelbottom=False, labelleft=False, bottom=False, left=False)  # 軸目盛りのラベルと線を消す
    plt.show()

## 学習
構築したGANのモデルを使って、学習を行います。  
バッチ正規化により、結果がどう変化するのか確認しましょう。  
学習には時間がかかりますので、なるべくGPUを使いましょう。

In [ ]:
batch_half = batch_size // 2

loss_record = np.zeros((n_learn, 3))
acc_record = np.zeros((n_learn, 2))

for i in range(n_learn):
    
    # ノイズから画像を生成しDiscriminatorを訓練
    g_noise = np.random.normal(0, 1, (batch_half, n_noize))
    g_imgs = generator.predict(g_noise)
    loss_fake, acc_fake = discriminator.train_on_batch(g_imgs, np.zeros((batch_half, 1)))
    loss_record[i][0] = loss_fake
    acc_record[i][0] = acc_fake

    # 本物の画像を使ってDiscriminatorを訓練
    rand_ids = np.random.randint(len(x_train), size=batch_half)
    real_imgs = x_train[rand_ids, :]
    loss_real, acc_real = discriminator.train_on_batch(real_imgs, np.ones((batch_half, 1)))
    loss_record[i][1] = loss_real
    acc_record[i][1] = acc_real

    # 結合したモデルによりGeneratorを訓練する
    c_noise = np.random.normal(0, 1, (batch_size, n_noize))
    loss_comb = combined.train_on_batch(c_noise, np.ones((batch_size, 1)))
    loss_record[i][2] = loss_comb

    # 一定間隔で生成された画像を表示
    if i % interval == 0:
        print ("n_learn:", i)
        print ("loss_fake:", loss_fake, "acc_fake:", acc_fake)
        print ("loss_real:", loss_real, "acc_real:", acc_real)
        print ("loss_comb:", loss_comb)

        generate_images(i)

学習がうまく進まないこともありますので、その際はモデル構築のセルから再実行しましょう。  

## 誤差と精度の推移
各誤差の収束と、Discriminatorの精度の向上を確認します。

In [ ]:
# 誤差の推移
n_plt_loss = 500
plt.plot(np.arange(n_plt_loss), loss_record[:n_plt_loss, 0], label="loss_fake")
plt.plot(np.arange(n_plt_loss), loss_record[:n_plt_loss, 1], label="loss_real")
plt.plot(np.arange(n_plt_loss), loss_record[:n_plt_loss, 2], label="loss_comb")
plt.legend()
plt.title("Loss")
plt.show()

# 精度の推移
n_plt_acc = 1000
plt.plot(np.arange(n_plt_acc), acc_record[:n_plt_acc, 0], label="acc_fake")
plt.plot(np.arange(n_plt_acc), acc_record[:n_plt_acc, 1], label="acc_real")
plt.legend()
plt.title("Accuracy")
plt.show()

# 解答例
以下は解答例です。  
どうしてもわからない時のみ、参考にしましょう。  


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LeakyReLU, BatchNormalization

# Generatorのネットワーク構築
generator = Sequential()
generator.add(Dense(256, input_shape=(n_noize,)))
generator.add(LeakyReLU(alpha=alpha))
generator.add(BatchNormalization())
generator.add(Dense(512))
generator.add(LeakyReLU(alpha=alpha)) 
generator.add(BatchNormalization())
generator.add(Dense(1024))
generator.add(LeakyReLU(alpha=alpha)) 
generator.add(BatchNormalization())
generator.add(Dense(img_size**2, activation="tanh"))
print(generator.summary())